# Social Network Recommendations

In this example, we're going to build a powerful social network predictive capability with Netpune ML. The techniques introduced here can be used to build predictions in other domains outside of social networks.

You can quickly setup the environment by using the Neptune ML AWS CloudFormation template: 
https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-quick-start.html

This code is extended base on Neptune ML code samples preconfigured using above CloudFormation. 

### People You May Know

Recommender systems are one of most widely adopted machine learning technologies in real world applications, ranging from social network to e-commerce platforms. In social network, one common use case is to recommend new friends to a user, based on user’s friendship with the others. Users that have common friends are likely to know each other, thus should have a higher score for recommendation system to propose if they are not yet connected.

### Setup

Before we begin, we'll clear any existing data from our Neptune cluster, using the cell magic `%%gremlin` and a subsequent drop query:

In [ ]:
%%gremlin

g.V().drop()

How do we know which Neptune cluster to access? The cell magics exposed by Neptune Notebooks use a configuration located by default under `~/graph_notebook_config.json` At the time of initialization of the Sagemaker instance, this configuration is generated using environment variables derived from the cluster being connected to. 

You can check the contents of the configuration in two ways. You can print the file itself, or you can look for the configuration being used by the notebook which you have opened.

In [ ]:
%%bash

cat ~/graph_notebook_config.json

### Create a Social Network

Next, we'll create a small social network. Note that the script below comprises a single statement. All the vertices and edges here will be created in the context of a single transaction.

In [ ]:
%%gremlin

g.
addV('User').property('name','Bill').property('interests', 'arts;comics;games;sports').
addV('User').property('name','Sarah').property('interests', 'arts').
addV('User').property('name','Ben').property('interests', 'electronics').
addV('User').property('name','Lucy').property('interests', 'electronics').
addV('User').property('name','Colin').property('interests', 'games;sports').
addV('User').property('name','Emily').property('interests', 'sports').
addV('User').property('name','Gordon').property('interests', 'sports').
addV('User').property('name','Kate').property('interests', 'arts').
addV('User').property('name','Peter').property('interests', 'games').
addV('User').property('name','Terry').property('interests', 'sports').
addV('User').property('name','Alistair').property('interests', 'arts;sports').
addV('User').property('name','Eve').property('interests', 'arts;electronics').
addV('User').property('name','Gary').property('interests', 'sports').
addV('User').property('name','Mary').property('interests', 'comics;games').
addV('User').property('name','Charlie').property('interests', 'games;electronics').
addV('User').property('name','Sue').property('interests', 'electronics').
addV('User').property('name','Arnold').property('interests', 'comics;games').
addV('User').property('name','Chloe').property('interests', 'sports').
addV('User').property('name','Henry').property('interests', 'arts;comics;games').
addV('User').property('name','Josie').property('interests', 'electronics').
V().hasLabel('User').has('name','Sarah').as('a').V().hasLabel('User').has('name','Bill').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Colin').as('a').V().hasLabel('User').has('name','Bill').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Terry').as('a').V().hasLabel('User').has('name','Bill').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Peter').as('a').V().hasLabel('User').has('name','Colin').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Kate').as('a').V().hasLabel('User').has('name','Ben').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Kate').as('a').V().hasLabel('User').has('name','Lucy').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Eve').as('a').V().hasLabel('User').has('name','Lucy').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Alistair').as('a').V().hasLabel('User').has('name','Kate').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Gary').as('a').V().hasLabel('User').has('name','Colin').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Gordon').as('a').V().hasLabel('User').has('name','Emily').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Alistair').as('a').V().hasLabel('User').has('name','Emily').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Terry').as('a').V().hasLabel('User').has('name','Gordon').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Alistair').as('a').V().hasLabel('User').has('name','Terry').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Gary').as('a').V().hasLabel('User').has('name','Terry').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Mary').as('a').V().hasLabel('User').has('name','Terry').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Henry').as('a').V().hasLabel('User').has('name','Alistair').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Sue').as('a').V().hasLabel('User').has('name','Eve').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Sue').as('a').V().hasLabel('User').has('name','Charlie').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Josie').as('a').V().hasLabel('User').has('name','Charlie').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Henry').as('a').V().hasLabel('User').has('name','Charlie').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Henry').as('a').V().hasLabel('User').has('name','Mary').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Mary').as('a').V().hasLabel('User').has('name','Gary').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Henry').as('a').V().hasLabel('User').has('name','Gary').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Chloe').as('a').V().hasLabel('User').has('name','Gary').addE('FRIEND').to('a').
V().hasLabel('User').has('name','Henry').as('a').V().hasLabel('User').has('name','Arnold').addE('FRIEND').to('a').
next()

This is what the network looks like:
    
<img src="https://s3.amazonaws.com/aws-neptune-customer-samples/neptune-sagemaker/images/03-social-network.png"/>

### Check the number of users in the graph 

In [ ]:
%%gremlin
g.V().groupCount().by(label).unfold()

### Check the number of relations among users

In [ ]:
%%gremlin
g.E().groupCount().by(label).unfold()

### Explore Henry's friends

In [ ]:
%%gremlin

g.V().hasLabel('User').has('name', 'Henry').both('FRIEND').groupCount().by('name')

### Explore Henry's interests

In [ ]:
%%gremlin

g.V().hasLabel('User').has('name', 'Henry').values('interests')

### Generate a recommendation by simple query

Let's now create a simple query to recommend for a specific user.

In the query below, we're finding the vertex that represents our user. We're then traversing `FRIEND` relationships (we don't care about relationship direction, so we're using `both()`) to find that user's immediate friends. We're then traversing another hop into the graph, looking for friends of those friends who _are not currently connected to our user.

We then count the paths to these candidate friends, and order the results based on the number of times we can reach a candidate via one of the user's immediate friends.

In [ ]:
%%gremlin

g.V().hasLabel('User').has('name', 'Henry').as('user').  
  both('FRIEND').aggregate('friends').  
  both('FRIEND').
    where(P.neq('user')).where(P.without('friends')).  
  groupCount().by('name').  
  order(Scope.local).by(values, Order.decr).
  next()

## Train your Graph Convolution Network with Amazon Neptune ML

Neptune ML uses graph neural network technology to automatically creates, trains, and applies ML models on your graph data. Neptune ML supports common graph prediction tasks such as node classification, node regression, edge classification and regression, and link prediction. 
It is powered by: 
- **Amazon Neptune:** a purpose-built, high-performance managed graph database, which is optimized for storing billions of relationships and querying the graph with milliseconds latency. Learn more at Overview of Amazon Neptune Features.
- **Amazon SageMaker:** a fully managed service that provides every developer and data scientist with the ability to prepare build, train, and deploy machine learning (ML) models quickly. 
- **Deep Graph Library (DGL):** an open-source, high performance and scalable Python package for deep learning on graphs. It provides fast and memory-efficient message passing primitives for training Graph Neural Networks. Neptune ML uses DGL to automatically choose and train the best ML model for your workload, enabling you to make ML-based predictions on graph data in hours instead of weeks. 

### Data export and configuration

The first step in our Neptune ML process is to export the graph data from the Neptune Cluster.

#### Setup for S3 bucket

In [ ]:
s3_bucket_uri="s3://(put-your-bucket-name-here-****)/neptune-ml-social-network-recommendation/"
# remove trailing slashes
s3_bucket_uri = s3_bucket_uri[:-1] if s3_bucket_uri.endswith('/') else s3_bucket_uri
s3_bucket_uri

In [ ]:
HOME_DIRECTORY = '~'

import os 
import json
import logging
def load_configuration():
    with open(os.path.expanduser(f'{HOME_DIRECTORY}/graph_notebook_config.json')) as f:
        data = json.load(f)
        host = data['host']
        port = data['port']
        if data['auth_mode'] == 'IAM':
            iam = True
        else:
            iam = False
    return host, port, iam


def get_host():
    host, port, iam = load_configuration()
    return host

In [ ]:
neptune_host = get_host()
neptune_host

In [ ]:
from urllib.parse import urlparse

def get_export_service_host():
    with open(os.path.expanduser(f'{HOME_DIRECTORY}/.bashrc')) as f:
        data = f.readlines()
        print(data)
    for d in data:
        if str.startswith(d, 'export NEPTUNE_EXPORT_API_URI'):
            parts = d.split('=')
            if len(parts) == 2:
                path = urlparse(parts[1].rstrip())
                return path.hostname + "/v1"
    logging.error(
        "Unable to determine the Neptune Export Service Endpoint. You will need to enter this or assign it manually.")
    return None

#### export_params

The first step in our Neptune ML process is to export the graph data from the Neptune Cluster. To do so, we need to specify the parameters for the data export and model configuration. Here is our example of export parameters. 

In export_params, we need to configure the basic setup such as the neptune host and output S3 path for exported data storage. The configuration specified in additionalParams is the type of machine learning task to perform. In this example, link prediction is optionally used to predict a particular edge type (User—FRIEND—User). If no target type is specified, Neptune ML will assume that the task is Link Prediction. The parameters also specify details about the data stored in our graph and how the machine learning model will interpret that data (we have “User” as node, and node property as “interests”). 

In [ ]:
export_params={ 
"command": "export-pg", 
"params": { "endpoint": neptune_host,
            "profile": "neptune_ml",
            "cloneCluster": False
            }, 
"outputS3Path": f'{s3_bucket_uri}/neptune-export',
"additionalParams": {
        "neptune_ml": {
          "version": "v2.0",
        "targets": [
            {
                "edge": ["User", "FRIEND", "User"],
                "type" : "link_prediction"
            }
         ],
         "features": [
            {
                "node": "User",
                "property": "interests",
                "type": "category",
                "separator": ";"
            }
         ]
        }
      },
"jobSize": "small"}

In [ ]:
%%neptune_ml export start --export-url {get_export_service_host()} --export-iam --wait --store-to export_results
${export_params}

Once the export job succeed, we will have the Neptune graph DB exported into CSV format and stored in an S3 bucket. There will be two types of files; nodes.csv and edges.csv. training-data-configuration.json: will also be generated which has configuration needed for Neptune ML to do model training. See [export data from Neptune for Neptune ML](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-data-export.html)


## Data processing

Neptune ML performs feature extraction and encoding as part of the data-processing steps. Common types of pre-processing of properties are: encoding categorical features through one-hot encoding, bucketing numerical features, or using word2vec to encode a string property or other free-form text property values.

In our example, we will simply use the property “interests”. Neptune ML encodes the values as multi-categorical. However, if such categorical value is complex, i.e. more than 3 words per node. Neptune ML infers the property type to be text and uses the text_word2vec encoding.

In [ ]:
# The training_job_name can be set to a unique value below, otherwise one will be auto generated
import time 
processing_job_name=f'social-link-prediction-processing-{int(time.time())}'

processing_params = f"""
--config-file-name training-data-configuration.json
--job-id {processing_job_name} 
--s3-input-uri {export_results['outputS3Uri']} 
--s3-processed-uri {str(s3_bucket_uri)}/preloading """

In [ ]:
%neptune_ml dataprocessing start --wait --store-to processing_results {processing_params}

At the end of this step, a DGL (Deep Graph library) graph is generated from the exported dataset for the model training step to use. Neptune ML automatically tune the model with Hyperparameter Optimization Tuning jobs defined in training-data-configuration.json. We can download and modify this file to tune the model’s hyperparameters, such as batch-size, num-hidden, num-epochs, dropout etc. 

See [Processing the graph data exported from Neptune for training](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-on-graphs-processing.html)



## Model training

The next step in the process is the automated training of the GNN model. The model training is done in two stages. The first stage uses a SageMaker Processing job to generate a model training strategy — a configuration set that specifies what type of model and model hyperparameter ranges will be used for the model training. 
Then, SageMaker hyperparameter tuning job will be launched. 

#### Important ! Change the batch size 

Neptune ML automatically tune the model with Hyperparameter Optimization Tuning jobs defined in training-data-configuration.json. Customer has the possibility to modify this file to tune the model according to the given parameters, such as batch_size, num-hidden, num-epochs, dropout etc. 

We illustrate how to change batch size. In our unconventional tiny network example here, it is required to change the batch size to prevent training job failure.  

In [ ]:
prcossing_location = processing_results['processingJob']['outputLocation']

In [ ]:
bucket_name, key_name = prcossing_location.replace("s3://", "").split("/", 1)

In [ ]:
import boto3

s3 = boto3.client('s3')
s3.download_file(bucket_name,key_name + '/model-hpo-configuration.json','model-hpo-configuration.json')

In [ ]:
!cat model-hpo-configuration.json

#### Replace batch-size as our network size is tiny 
                {
                    "param": "batch-size",
                    "range": [
                        2,
                        4
                    ],
                    "inc_strategy": "power2",
                    "type": "int",
                    "default": 2
                },

In [ ]:
s3.upload_file('model-hpo-configuration.json', bucket_name, key_name + '/model-hpo-configuration.json')

The SageMaker Hyperparameter Tuning Optimization job runs a pre-specified number of model training job trials on the processed data, try different hyperparameter combinaisons according to **model-hpo-configuration.json**, and stores the model artifacts generated by the training in the output S3 location. 

In [ ]:
training_job_name=f'social-link-prediction-{int(time.time())}'

training_params=f"""
--job-id {training_job_name} 
--data-processing-id {processing_job_name} 
--instance-type ml.c5.xlarge
--s3-output-uri {str(s3_bucket_uri)}/training """

In [ ]:
%neptune_ml training start --wait --store-to training_results {training_params}

## Create an inference endpoint in Amazon SageMaker

Now that the graph representation is learned, we can deploy the learned model behind an endpoint to perform inference requests.

In [ ]:
endpoint_params=f"""
--job-id {training_job_name} 
--model-job-id {training_job_name}"""

In [ ]:
%neptune_ml endpoint create --wait --store-to endpoint_results {endpoint_params}

In [ ]:
endpoint_name=endpoint_results['endpoint']['name']

## Query the machine learning model using Gremlin

Once the endpoint is ready, we can use it for graph inference queries. In our example, we can now check the friends recommendation with Neptune ML on User “Henry”. It requires almost the exact same syntax to traverse the edge, and list the other User that are connected to Henry through FRIEND connection.

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint_name}").
      V().hasLabel('User').has('name', 'Henry').
        out('FRIEND').with("Neptune#ml.prediction").hasLabel('User').values('name')

Here is another sample prediction query, used to predict the top eight users that are most likely to connect with Henry.

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint_name}").with("Neptune#ml.limit",8).
    V().hasLabel('User').has('name', 'Henry').out('FRIEND').with("Neptune#ml.prediction").hasLabel('User').values('name')

### Delete the endpoint 

Now that you have completed this walkthrough you have created a Sagemaker endpoint which is currently running and will incur the standard charges.  If you are done trying out Neptune ML and would like to avoid these recurring costs, run the cell below to delete the inference endpoint.

In [ ]:
import boto3
sm_boto3 = boto3.client('sagemaker')
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

## Model transform or retraining when graph data changed

In the scenarios where you have continuously changing graphs, you may need to update ML predictions with the newest graph data. The generated model artifacts after training are directly tied to the training graph which means that the inference endpoint needs to be updated once the entities in the original training graph changes. 

However, you don’t need to retrain the whole model in order to make predictions on the updated graph. With incremental model inference workflow, you only need to export the data from Neptune DB, incremental data preprocessing, model transform and update the inference endpoint. The model-transform step takes the trained model from the main workflow and the results of the incremental data preprocessing step as inputs, and output new model artifact to use for inference. This new model artifact has the up-to-date graph. 

See more Neptune ML implementation details at [Generating new model artifacts](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-model-artifacts.html)